# Lab | Revisiting Machine Learning Case Study

Complete the following steps on the categorical columns in the dataset:

In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

### reading the file

In [3]:
df = pd.read_csv('learningSet.csv')
df.head()

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,GRI,0,IL,61081,,,3712,0,,...,0.0,0,L,4,E,X,X,X,39.0,C
1,9401,BOA,1,CA,91326,,,5202,0,,...,0.0,0,L,2,G,X,X,X,1.0,A
2,9001,AMH,1,NC,27017,,,0,0,,...,0.0,1,L,4,E,X,X,X,60.0,C
3,8701,BRY,0,CA,95953,,,2801,0,,...,0.0,1,L,4,E,X,X,X,41.0,C
4,8601,,0,FL,33176,,,2001,0,X,...,0.0,1,L,2,F,X,X,X,26.0,A


In [4]:
df.shape

(95412, 481)


#### Check for null values in all the columns

In [5]:
df.isna().sum()

ODATEDW       0
OSOURCE       0
TCODE         0
STATE         0
ZIP           0
           ... 
MDMAUD_R      0
MDMAUD_F      0
MDMAUD_A      0
CLUSTER2    132
GEOCODE2    132
Length: 481, dtype: int64

In [6]:
df_nulls = pd.DataFrame(df.isna().sum()*100/len(df), columns=['percentage'])
df_nulls

,percentage
ODATEDW,0.000000
OSOURCE,0.000000
TCODE,0.000000
STATE,0.000000
ZIP,0.000000
...,...
MDMAUD_R,0.000000
MDMAUD_F,0.000000
MDMAUD_A,0.000000
CLUSTER2,0.138347


#### Checking the categorical columns

In [8]:
categorical = df.select_dtypes(np.object)
categorical.head()

,OSOURCE,STATE,ZIP,MAILCODE,PVASTATE,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,...,RFA_21,RFA_22,RFA_23,RFA_24,RFA_2R,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,GRI,IL,61081,,,0,,,,,...,S4E,S4E,S4E,S4E,L,E,X,X,X,C
1,BOA,CA,91326,,,0,,,,,...,N1E,N1E,,F1E,L,G,X,X,X,A
2,AMH,NC,27017,,,0,,,,,...,,S4D,S4D,S3D,L,E,X,X,X,C
3,BRY,CA,95953,,,0,,,,,...,A1D,A1D,,,L,E,X,X,X,C
4,,FL,33176,,,0,X,X,,,...,A3D,I4E,A3D,A3D,L,F,X,X,X,A


In [9]:
n = categorical.isnull().sum().to_frame()
n.columns = ['num_of_nulls']
n[n.num_of_nulls > 0] # to see the columns that have more than 0 null values

,num_of_nulls
GEOCODE2,132


#### Exclude the following variables by looking at the definitions. Create a new empty list called drop_list. We will append this list and then drop all the columns in this list later:

* OSOURCE - symbol definitions not provided, too many categories
* ZIP CODE - we are including state already

In [29]:
categorical['OSOURCE'].value_counts()

MBC    4539
SYN    3563
AML    3430
BHG    3324
IMP    2986
       ... 
MDD       1
NRM       1
HDP       1
CRP       1
VIC       1
Name: OSOURCE, Length: 896, dtype: int64

In [48]:
categorical['ZIP'].value_counts()

85351     61
92653     59
85710     54
95608     50
60619     45
          ..
70058-     1
93024      1
96306      1
30060-     1
32776-     1
Name: ZIP, Length: 19938, dtype: int64

there are too many unique values in these particular columns, so we can drop them. 

In [50]:
categorical = categorical.drop(['OSOURCE', 'ZIP'], axis =1 )
categorical.head()


,STATE,MAILCODE,PVASTATE,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,MDMAUD,DOMAIN,...,RFA_21,RFA_22,RFA_23,RFA_24,RFA_2R,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,IL,,,0,,,,,XXXX,T2,...,S4E,S4E,S4E,S4E,L,E,X,X,X,C
1,CA,,,0,,,,,XXXX,S1,...,N1E,N1E,,F1E,L,G,X,X,X,A
2,NC,,,0,,,,,XXXX,R2,...,,S4D,S4D,S3D,L,E,X,X,X,C
3,CA,,,0,,,,,XXXX,R2,...,A1D,A1D,,,L,E,X,X,X,C
4,FL,,,0,X,X,,,XXXX,S2,...,A3D,I4E,A3D,A3D,L,F,X,X,X,A


#### Identify columns that over 85% missing values

there are 2 ways to calculate: 

In [11]:
#1st way
all_nulls = pd.DataFrame(df.isna().sum()*100/len(df))
all_nulls.columns = ['nulls']
len(all_nulls[all_nulls.nulls > 85])

25

In [12]:
#2nd way
most_85_missing_cols = set(df.columns[np.sum(df.isnull())/df.shape[0] > .85])
len(most_85_missing_cols)


25

#### Remove those columns from the dataframe

In [21]:
df2 = df.copy() # just to save a copy

#for col in df2:
 #   df2.drop(most_85_missing_cols[col])
df2.drop(['RAMNT_21', 'RDATE_4', 'RAMNT_17', 'RAMNT_5', 'RAMNT_10', 'RAMNT_4', 'NUMCHLD', 'RDATE_15', 'RDATE_21', 'RDATE_17', 'RDATE_20', 'RAMNT_3', 'RAMNT_13', 'RAMNT_20', 'RDATE_3', 'RDATE_7', 'RAMNT_7', 'RAMNT_6', 'RDATE_6', 'RDATE_10', 'RDATE_5', 'RDATE_23', 'RAMNT_15', 'RAMNT_23', 'RDATE_13'], axis =1, inplace=True)
df2.head()

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,GRI,0,IL,61081,,,3712,0,,...,0.0,0,L,4,E,X,X,X,39.0,C
1,9401,BOA,1,CA,91326,,,5202,0,,...,0.0,0,L,2,G,X,X,X,1.0,A
2,9001,AMH,1,NC,27017,,,0,0,,...,0.0,1,L,4,E,X,X,X,60.0,C
3,8701,BRY,0,CA,95953,,,2801,0,,...,0.0,1,L,4,E,X,X,X,41.0,C
4,8601,,0,FL,33176,,,2001,0,X,...,0.0,1,L,2,F,X,X,X,26.0,A


#### Reduce the number of categories in the column GENDER. The column should only have either "M" for males, "F" for females, and "other" for all the rest

In [23]:
categorical['GENDER'].value_counts()

F    51277
M    39094
      2957
U     1715
J      365
C        2
A        2
Name: GENDER, dtype: int64

In [27]:
categorical['GENDER'].replace({' ' : 'Other', 'U' : 'Other', 'C': 'Other', 'A': 'Other', 'J': 'Other'}, inplace=True) 
categorical['GENDER'].value_counts()

F        51277
M        39094
Other     5041
Name: GENDER, dtype: int64